# Planeación de rutas para la exploración de Marte

- Diseño de agentes inteligentes
- Profesor: Juan Emmanuel Martínez Ledesma

Equipo 7:

| Nombre | Matrícula |
| ----- | ------ |
| Juan Pablo Echeagaray González | A00830646 |

- 8 de marzo del 2022


## Descripción del entregable?

In [30]:
# Basic dependencies
from simpleai.search import SearchProblem
from simpleai.search import breadth_first, uniform_cost, depth_first, astar
import numpy as np
import matplotlib.pyplot as plt
import struct
import copy
# For plotting
from matplotlib.colors import LightSource
from skimage.transform import downscale_local_mean
import plotly.graph_objects as px


### Características del problema

In [31]:
SCALE = 10.0174
mars_map = np.load('mars_map.npy')
NR, NC = mars_map.shape

# Start and goal positions
x0, y0 = 2860, 6300
xf, yf = 3150, 6800
MAX_DIFF = 0.25
UNTRAVERSABLE = -2


In [32]:
def pos_to_index(pos: list[int]) -> list[int]:
    nr = NR - round(pos[0] / SCALE)
    nc = NC - round(pos[1] / SCALE)
    return [nr, nc]


def coord_to_state(coord: list[int]) -> str:
    return f'{coord[0]},{coord[1]}'


def state_to_coord(state: str) -> list[int]:
    return [int(x) for x in state.split(',')]


In [33]:
coord = [0, 0]
mars_map[0, 0]

-1.0

In [34]:
START = pos_to_index((x0, y0))
GOAL = pos_to_index((xf, yf))


### Implementación del problema

In [35]:
class mars_route_finding(SearchProblem):

    def __init__(self, initial_state: str, goal_state: str, mars_map: np.array, max_height: float, not_traversable: int):

        self.mars_map = mars_map
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.max_height = max_height
        self.not_traversable = not_traversable

        print("Initial state: ", self.initial_state)
        print("Goal state: ", self.goal_state)

        SearchProblem.__init__(self, initial_state)

    def actions(self, state):
        mars_map = self.mars_map
        actions = []

        # Up, Down, Left, Right, Up-Left, Up-Right, Down-Left, Down-Right
        pos_directions = ['U', 'D', 'L', 'R', 'UL', 'UR', 'DL', 'DR']
        current_pos = tuple(state_to_coord(state))

        for dir in pos_directions:
            if dir == 'U':
                next_pos = tuple([current_pos[0] + 1, current_pos[1]])
                print(mars_map[current_pos] - mars_map[next_pos], mars_map[next_pos])
                if (np.abs(mars_map[current_pos] - mars_map[next_pos]) <= self.max_height) and (mars_map[next_pos] != self.not_traversable):
                    print('Estoy yendo para arriba')
                    actions.append(dir)
                else:
                    print('no entró el up')
                    continue
            elif dir == 'D':
                next_pos = tuple([current_pos[0] - 1, current_pos[1]])
                if (np.abs(mars_map[current_pos] - mars_map[next_pos]) <= self.max_height) and (mars_map[next_pos] != self.not_traversable):
                    print('Estoy yendo pa abajo')
                    actions.append(dir)
                else:
                    print('no entró el down')
                    continue
            elif dir == 'L':
                next_pos = tuple([current_pos[0], current_pos[1] - 1])
                print(np.abs(mars_map[current_pos] - mars_map[next_pos]))
                if (np.abs(mars_map[current_pos] - mars_map[next_pos]) <= self.max_height) and (mars_map[next_pos] != self.not_traversable):
                    print('Estoy yendo pa la izquierda')
                    actions.append(dir)
                else:
                    print('no entró el left')
                    continue
            elif dir == 'R':
                next_pos = tuple([current_pos[0], current_pos[1] + 1])
                if (np.abs(mars_map[current_pos] - mars_map[next_pos]) <= self.max_height) and (mars_map[next_pos] != self.not_traversable):
                    print('Estoy yendo pa la derecha')
                    actions.append(dir)
                else:
                    print('no entró el right')
                    continue
            elif dir == 'UL':
                next_pos = tuple([current_pos[0] + 1, current_pos[1] - 1])
                if (np.abs(mars_map[current_pos] - mars_map[next_pos]) <= self.max_height) and (mars_map[next_pos] != self.not_traversable):
                    print('Estoy yendo pa arriba izquierda')
                    actions.append(dir)
                else:
                    print('no entró el up-left')
                    continue
            elif dir == 'UR':
                next_pos = tuple([current_pos[0] + 1, current_pos[1] + 1])
                if (np.abs(mars_map[current_pos] - mars_map[next_pos]) <= self.max_height) and (mars_map[next_pos] != self.not_traversable):
                    print('Estoy yendo pa arriba derecha')
                    actions.append(dir)
                else:
                    print('no entró el up-right')
                    continue
            elif dir == 'DL':
                next_pos = tuple([current_pos[0] - 1, current_pos[1] - 1])
                if (np.abs(mars_map[current_pos] - mars_map[next_pos]) <= self.max_height) and (mars_map[next_pos] != self.not_traversable):
                    print('Estoy yendo pa abajo izquierda')
                    actions.append(dir)
                else:
                    print('no entró el down-left')
                    continue
            elif dir == 'DR':
                next_pos = tuple([current_pos[0] - 1, current_pos[1] + 1])
                if (np.abs(mars_map[current_pos] - mars_map[next_pos]) <= self.max_height) and (mars_map[next_pos] != self.not_traversable):
                    print('Estoy yendo pa abajo derecha')
                    actions.append(dir)
                else:
                    print('no entró el down-right')
                    continue
            else:
                continue
        return actions

    def result(self, state, action):
        current_pos = state_to_coord(state)

        if action == 'U':
            next_pos = [current_pos[0] + 100, current_pos[1]]

        elif action == 'D':
            next_pos = [current_pos[0] - 100, current_pos[1]]

        elif action == 'L':
            next_pos = [current_pos[0], current_pos[1] - 100]

        elif action == 'R':
            next_pos = [current_pos[0], current_pos[1] + 100]

        elif action == 'UL':
            next_pos = [current_pos[0] + 100, current_pos[1] - 100]

        elif action == 'UR':
            next_pos = [current_pos[0] + 100, current_pos[1] + 100]

        elif action == 'DL':
            next_pos = [current_pos[0] - 100, current_pos[1] - 100]

        elif action == 'DR':
            next_pos = [current_pos[0] - 100, current_pos[1] + 100]
        else:
            print('Invalid action')

        return coord_to_state(tuple(next_pos))

    def is_goal(self, state):
        current_pos = state_to_coord(state)

        return current_pos[0] == self.goal_state[0] and current_pos[1] == self.goal_state[1]

    def cost(self, state, action, state2):
        # This smells of a linear combination where values for a and b must be found such that the search algorithm
        # performs the best
        alpha, beta = 1, 1

        current_pos = state_to_coord(state)
        next_position = state_to_coord(state2)

        distance = np.sqrt(
            (current_pos[0] - next_position[0])**2 + (current_pos[1] - next_position[1])**2)
        height_difference = abs(
            self.mars_map[current_pos] - self.mars_map[next_position])

        return alpha * distance + beta * height_difference

    def heuristic(self, state):
        current_pos = state_to_coord(state)
        goal_pos = state_to_coord(self.goal_state)
        distance = np.sqrt((current_pos[0] - goal_pos[0])**2 + (current_pos[1] - goal_pos[1])**2)
        return distance


In [36]:
my_problem = mars_route_finding(coord_to_state(
    START), coord_to_state(GOAL), mars_map, 100000 * MAX_DIFF, UNTRAVERSABLE)


Initial state:  1529,127
Goal state:  1501,77


In [37]:
result = breadth_first(my_problem)
print(result)

0.0 -1.0
Estoy yendo para arriba
Estoy yendo pa abajo
0.0
Estoy yendo pa la izquierda
Estoy yendo pa la derecha
Estoy yendo pa arriba izquierda
Estoy yendo pa arriba derecha
Estoy yendo pa abajo izquierda
Estoy yendo pa abajo derecha
0.0 -1.0
Estoy yendo para arriba
Estoy yendo pa abajo
0.0
Estoy yendo pa la izquierda
Estoy yendo pa la derecha
Estoy yendo pa arriba izquierda
Estoy yendo pa arriba derecha
Estoy yendo pa abajo izquierda
Estoy yendo pa abajo derecha
0.0 -1.0
Estoy yendo para arriba
Estoy yendo pa abajo
0.0
Estoy yendo pa la izquierda
Estoy yendo pa la derecha
Estoy yendo pa arriba izquierda
Estoy yendo pa arriba derecha
Estoy yendo pa abajo izquierda
Estoy yendo pa abajo derecha
0.0 -1.0
Estoy yendo para arriba
Estoy yendo pa abajo
0.0
Estoy yendo pa la izquierda
Estoy yendo pa la derecha
Estoy yendo pa arriba izquierda
Estoy yendo pa arriba derecha
Estoy yendo pa abajo izquierda
Estoy yendo pa abajo derecha
0.7807690429687568 158.66283203125013
Estoy yendo para arriba
Es

IndexError: index 1829 is out of bounds for axis 0 with size 1815

In [38]:
result = uniform_cost(my_problem)
print(result)

0.0 -1.0
Estoy yendo para arriba
Estoy yendo pa abajo
0.0
Estoy yendo pa la izquierda
Estoy yendo pa la derecha
Estoy yendo pa arriba izquierda
Estoy yendo pa arriba derecha
Estoy yendo pa abajo izquierda
Estoy yendo pa abajo derecha


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [39]:
result = depth_first(my_problem)
print(result)

0.0 -1.0
Estoy yendo para arriba
Estoy yendo pa abajo
0.0
Estoy yendo pa la izquierda
Estoy yendo pa la derecha
Estoy yendo pa arriba izquierda
Estoy yendo pa arriba derecha
Estoy yendo pa abajo izquierda
Estoy yendo pa abajo derecha
-0.002023925781259095 198.61049560546886
Estoy yendo para arriba
Estoy yendo pa abajo
0.36398437500000114
Estoy yendo pa la izquierda
Estoy yendo pa la derecha
Estoy yendo pa arriba izquierda
Estoy yendo pa arriba derecha
Estoy yendo pa abajo izquierda
Estoy yendo pa abajo derecha
-0.06905517578124432 130.5552465820314
Estoy yendo para arriba
Estoy yendo pa abajo
0.5186303710937352
Estoy yendo pa la izquierda
Estoy yendo pa la derecha
Estoy yendo pa arriba izquierda
Estoy yendo pa arriba derecha
Estoy yendo pa abajo izquierda
Estoy yendo pa abajo derecha
1.4463085937499898 136.1784399414064
Estoy yendo para arriba
Estoy yendo pa abajo
2.7431811523437375
Estoy yendo pa la izquierda
Estoy yendo pa la derecha
Estoy yendo pa arriba izquierda
Estoy yendo pa arr

IndexError: index 827 is out of bounds for axis 1 with size 756

In [17]:
print(np.unique(mars_map)[1] - np.unique(mars_map)[2])

-5.429687498690293e-05
